# Türkçe Veri Seti Hazırlama

Bu notebook Hugging Face'den iki farklı veri setini indirip, temizleyip, kalite kontrolü yaparak 200K örneklik yüksek kaliteli Türkçe veri seti oluşturur.

- **muspdf**: Kalite eşiği > %88
- **CulturaX**: Kalite eşiği > %95

In [ ]:
# Gerekli kütüphaneleri yükle
!pip install -q datasets transformers langdetect ftfy pandas numpy tqdm
!pip install -q huggingface_hub

In [ ]:
import os
import re
import ftfy
import pandas as pd
import numpy as np
from tqdm import tqdm
from datasets import load_dataset, Dataset, concatenate_datasets
from langdetect import detect, LangDetectException
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Google Colab ortam kontrolü
try:
    from google.colab import drive
    IN_COLAB = True
    print("✅ Google Colab ortamında çalışıyor")
except ImportError:
    IN_COLAB = False
    print("💻 Lokal ortamda çalışıyor")

In [ ]:
# Google Drive'ı bağla (isteğe bağlı)
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    OUTPUT_DIR = '/content/drive/MyDrive/turkish_dataset'
else:
    OUTPUT_DIR = './turkish_dataset'

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📁 Çıktı dizini: {OUTPUT_DIR}")

## Veri Temizleme ve Kalite Kontrol Fonksiyonları

In [ ]:
class TurkishTextCleaner:
    """Türkçe metinler için temizleme ve kalite kontrol sınıfı"""
    
    def __init__(self):
        # Türkçe karakterler
        self.turkish_chars = set('abcçdefgğhıijklmnoöprsştuüvyzABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ')
        self.turkish_vowels = set('aeıioöuüAEIİOÖUÜ')
        
        # Temizleme regex'leri
        self.url_pattern = re.compile(r'https?://\S+|www\.\S+')
        self.email_pattern = re.compile(r'\S+@\S+')
        self.html_pattern = re.compile(r'<[^>]+>')
        self.multiple_spaces = re.compile(r'\s+')
        self.multiple_newlines = re.compile(r'\n{3,}')
        
    def clean_text(self, text: str) -> str:
        """Metni temizle"""
        if not text or not isinstance(text, str):
            return ""
        
        # Encoding düzeltmesi
        text = ftfy.fix_text(text)
        
        # URL, email ve HTML temizleme
        text = self.url_pattern.sub(' ', text)
        text = self.email_pattern.sub(' ', text)
        text = self.html_pattern.sub(' ', text)
        
        # Özel karakterleri temizle (Türkçe karakterleri koru)
        cleaned_chars = []
        for char in text:
            if char.isalnum() or char.isspace() or char in self.turkish_chars or char in '.,!?;:"-':
                cleaned_chars.append(char)
            else:
                cleaned_chars.append(' ')
        text = ''.join(cleaned_chars)
        
        # Fazla boşlukları temizle
        text = self.multiple_spaces.sub(' ', text)
        text = self.multiple_newlines.sub('\n\n', text)
        
        return text.strip()
    
    def is_turkish(self, text: str) -> bool:
        """Metnin Türkçe olup olmadığını kontrol et"""
        try:
            lang = detect(text[:500])  # İlk 500 karaktere bak
            return lang == 'tr'
        except (LangDetectException, Exception):
            # Dil algılanamadıysa Türkçe karakterlere bak
            return self.calculate_turkish_char_ratio(text) > 0.05
    
    def calculate_turkish_char_ratio(self, text: str) -> float:
        """Türkçe özel karakter oranını hesapla"""
        if not text:
            return 0.0
        
        turkish_specific = set('çğıöşüÇĞİÖŞÜ')
        char_count = sum(1 for c in text if c.isalpha())
        if char_count == 0:
            return 0.0
        
        turkish_count = sum(1 for c in text if c in turkish_specific)
        return turkish_count / char_count
    
    def calculate_quality_score(self, text: str) -> float:
        """Metin kalitesi skorunu hesapla (0-100)"""
        if not text:
            return 0.0
        
        scores = []
        
        # 1. Uzunluk skoru (100-5000 karakter ideal)
        text_len = len(text)
        if 100 <= text_len <= 5000:
            len_score = 100
        elif text_len < 100:
            len_score = text_len
        else:
            len_score = max(0, 100 - (text_len - 5000) / 100)
        scores.append(len_score * 0.2)
        
        # 2. Türkçe karakter oranı
        turkish_ratio = self.calculate_turkish_char_ratio(text)
        scores.append(min(turkish_ratio * 500, 100) * 0.3)  # %5 Türkçe karakter = 25 puan
        
        # 3. Kelime çeşitliliği
        words = text.lower().split()
        if len(words) > 0:
            unique_ratio = len(set(words)) / len(words)
            scores.append(unique_ratio * 100 * 0.2)
        else:
            scores.append(0)
        
        # 4. Cümle yapısı (noktalama işaretleri)
        punct_count = sum(1 for c in text if c in '.!?')
        if len(words) > 0:
            punct_ratio = punct_count / (len(words) / 10)  # Her 10 kelimede 1 noktalama beklenir
            scores.append(min(punct_ratio * 100, 100) * 0.15)
        else:
            scores.append(0)
        
        # 5. Büyük/küçük harf dengesi
        alpha_chars = [c for c in text if c.isalpha()]
        if alpha_chars:
            upper_ratio = sum(1 for c in alpha_chars if c.isupper()) / len(alpha_chars)
            if 0.02 <= upper_ratio <= 0.15:  # %2-%15 arası büyük harf ideal
                scores.append(100 * 0.15)
            else:
                scores.append(50 * 0.15)
        else:
            scores.append(0)
        
        return sum(scores)
    
    def process_text(self, text: str) -> Optional[Dict]:
        """Metni işle ve sonuçları döndür"""
        # Temizle
        cleaned = self.clean_text(text)
        
        # Boş veya çok kısa metinleri ele
        if not cleaned or len(cleaned) < 50:
            return None
        
        # Türkçe kontrolü
        if not self.is_turkish(cleaned):
            return None
        
        # Kalite skoru hesapla
        quality_score = self.calculate_quality_score(cleaned)
        
        return {
            'text': cleaned,
            'quality_score': quality_score,
            'length': len(cleaned),
            'word_count': len(cleaned.split())
        }

## Adım 1: muspdf Veri Setini İndir ve İşle

In [ ]:
# muspdf veri setini indir
print("📥 muspdf veri seti indiriliyor...")
try:
    # Huseyin/muspdf veri setini yükle
    muspdf_dataset = load_dataset("Huseyin/muspdf", split="train", streaming=True)
    
    # İlk 500K veriyi al (bellek yönetimi için)
    muspdf_data = []
    for i, item in enumerate(tqdm(muspdf_dataset, desc="muspdf yükleniyor", total=500000)):
        if i >= 500000:
            break
        # Veri yapısına göre text alanını al
        text = item.get('text', '') or item.get('content', '') or str(item)
        muspdf_data.append({'text': text})
    
    print(f"✅ {len(muspdf_data)} muspdf örneği yüklendi")
except Exception as e:
    print(f"⚠️ muspdf yüklenemedi: {e}")
    print("Alternatif veri seti kullanılıyor...")
    # Alternatif olarak başka bir Türkçe veri seti kullan
    muspdf_dataset = load_dataset("uonlp/CulturaX", "tr", split="train", streaming=True)
    muspdf_data = []
    for i, item in enumerate(tqdm(muspdf_dataset, desc="Alternatif veri yükleniyor", total=100000)):
        if i >= 100000:
            break
        text = item.get('text', '') or item.get('content', '') or str(item)
        muspdf_data.append({'text': text})

In [ ]:
# muspdf verilerini temizle ve kalite kontrolü yap
print("🧹 muspdf verileri temizleniyor ve kalite kontrolü yapılıyor...")

cleaner = TurkishTextCleaner()
muspdf_cleaned = []

for item in tqdm(muspdf_data, desc="muspdf işleniyor"):
    result = cleaner.process_text(item['text'])
    if result and result['quality_score'] >= 88:  # %88 üstü kalite
        muspdf_cleaned.append(result)

print(f"✅ {len(muspdf_cleaned)} yüksek kaliteli muspdf örneği seçildi")
print(f"📊 Ortalama kalite skoru: {np.mean([x['quality_score'] for x in muspdf_cleaned]):.2f}")

## Adım 2: CulturaX Veri Setini İndir ve İşle

In [ ]:
# CulturaX Türkçe veri setini indir
print("📥 CulturaX Türkçe veri seti indiriliyor...")

try:
    # CulturaX'dan Türkçe verileri yükle (streaming mode)
    culturax_dataset = load_dataset("uonlp/CulturaX", "tr", split="train", streaming=True)
    
    # 300K veri al
    culturax_data = []
    for i, item in enumerate(tqdm(culturax_dataset, desc="CulturaX yükleniyor", total=300000)):
        if i >= 300000:
            break
        text = item.get('text', '') or item.get('content', '') or str(item)
        culturax_data.append({'text': text})
    
    print(f"✅ {len(culturax_data)} CulturaX örneği yüklendi")
except Exception as e:
    print(f"⚠️ CulturaX yüklenemedi: {e}")
    culturax_data = []

In [ ]:
# CulturaX verilerini temizle ve kalite kontrolü yap
print("🧹 CulturaX verileri temizleniyor ve kalite kontrolü yapılıyor...")

culturax_cleaned = []

for item in tqdm(culturax_data, desc="CulturaX işleniyor"):
    result = cleaner.process_text(item['text'])
    if result and result['quality_score'] >= 95:  # %95 üstü kalite
        culturax_cleaned.append(result)

print(f"✅ {len(culturax_cleaned)} yüksek kaliteli CulturaX örneği seçildi")
if culturax_cleaned:
    print(f"📊 Ortalama kalite skoru: {np.mean([x['quality_score'] for x in culturax_cleaned]):.2f}")

## Adım 3: Veri Setlerini Birleştir ve 200K Örnek Seç

In [ ]:
# İki veri setini birleştir
print("🔗 Veri setleri birleştiriliyor...")

all_data = muspdf_cleaned + culturax_cleaned
print(f"📊 Toplam temizlenmiş veri: {len(all_data)} örnek")

# Kalite skoruna göre sırala
all_data_sorted = sorted(all_data, key=lambda x: x['quality_score'], reverse=True)

# En iyi 200K örneği seç
final_dataset = all_data_sorted[:200000]
print(f"✅ En yüksek kaliteli {len(final_dataset)} örnek seçildi")
print(f"📊 Final veri seti ortalama kalite skoru: {np.mean([x['quality_score'] for x in final_dataset]):.2f}")
print(f"📊 Min kalite skoru: {min([x['quality_score'] for x in final_dataset]):.2f}")
print(f"📊 Max kalite skoru: {max([x['quality_score'] for x in final_dataset]):.2f}")

In [ ]:
# Veri seti istatistikleri
print("\n📈 Veri Seti İstatistikleri:")
print("=" * 50)

lengths = [x['length'] for x in final_dataset]
word_counts = [x['word_count'] for x in final_dataset]
quality_scores = [x['quality_score'] for x in final_dataset]

print(f"📝 Ortalama metin uzunluğu: {np.mean(lengths):.0f} karakter")
print(f"📝 Medyan metin uzunluğu: {np.median(lengths):.0f} karakter")
print(f"📝 Ortalama kelime sayısı: {np.mean(word_counts):.0f} kelime")
print(f"📊 Kalite skoru dağılımı:")
print(f"   - Q1 (25%): {np.percentile(quality_scores, 25):.2f}")
print(f"   - Q2 (50%): {np.percentile(quality_scores, 50):.2f}")
print(f"   - Q3 (75%): {np.percentile(quality_scores, 75):.2f}")

## Adım 4: Veri Setini Kaydet

In [ ]:
# Hugging Face Dataset formatına dönüştür
print("💾 Veri seti kaydediliyor...")

# Dataset oluştur
dataset_dict = {
    'text': [item['text'] for item in final_dataset],
    'quality_score': [item['quality_score'] for item in final_dataset],
    'length': [item['length'] for item in final_dataset],
    'word_count': [item['word_count'] for item in final_dataset]
}

hf_dataset = Dataset.from_dict(dataset_dict)

# Farklı formatlarda kaydet
# 1. Hugging Face format
hf_dataset.save_to_disk(os.path.join(OUTPUT_DIR, 'turkish_200k_dataset'))
print(f"✅ Hugging Face formatında kaydedildi: {OUTPUT_DIR}/turkish_200k_dataset")

# 2. CSV formatı
df = pd.DataFrame(dataset_dict)
csv_path = os.path.join(OUTPUT_DIR, 'turkish_200k_dataset.csv')
df.to_csv(csv_path, index=False)
print(f"✅ CSV formatında kaydedildi: {csv_path}")

# 3. JSONL formatı (her satır bir JSON objesi)
jsonl_path = os.path.join(OUTPUT_DIR, 'turkish_200k_dataset.jsonl')
df.to_json(jsonl_path, orient='records', lines=True, force_ascii=False)
print(f"✅ JSONL formatında kaydedildi: {jsonl_path}")

# 4. Parquet formatı (daha verimli depolama)
parquet_path = os.path.join(OUTPUT_DIR, 'turkish_200k_dataset.parquet')
df.to_parquet(parquet_path)
print(f"✅ Parquet formatında kaydedildi: {parquet_path}")

In [ ]:
# Örnek metinleri göster
print("\n📄 Örnek Metinler:")
print("=" * 50)

for i in range(min(3, len(final_dataset))):
    print(f"\n🔹 Örnek {i+1}:")
    print(f"Kalite Skoru: {final_dataset[i]['quality_score']:.2f}")
    print(f"Uzunluk: {final_dataset[i]['length']} karakter")
    print(f"Kelime Sayısı: {final_dataset[i]['word_count']} kelime")
    print(f"Metin (ilk 200 karakter): {final_dataset[i]['text'][:200]}...")
    print("-" * 40)

In [ ]:
# Veri setini yükleme testi
print("\n🔄 Veri seti yükleme testi...")

try:
    # Kaydedilen veri setini tekrar yükle
    loaded_dataset = Dataset.load_from_disk(os.path.join(OUTPUT_DIR, 'turkish_200k_dataset'))
    print(f"✅ Veri seti başarıyla yüklendi: {len(loaded_dataset)} örnek")
    print(f"Sütunlar: {loaded_dataset.column_names}")
    print(f"İlk örnek: {loaded_dataset[0]['text'][:100]}...")
except Exception as e:
    print(f"❌ Veri seti yüklenemedi: {e}")

## Kullanım Örneği

In [ ]:
# Veri setini kullanma örneği
print("\n💡 Veri Setini Kullanma Örneği:")
print("=" * 50)

print("""
# Veri setini yüklemek için:
from datasets import load_from_disk

# Hugging Face formatından yükle
dataset = load_from_disk('turkish_200k_dataset')

# veya CSV'den yükle
import pandas as pd
df = pd.read_csv('turkish_200k_dataset.csv')

# Model eğitimi için kullan
for example in dataset:
    text = example['text']
    quality = example['quality_score']
    # Model eğitim kodu...
""")

print("\n✨ Veri seti hazırlama tamamlandı!")
print(f"📁 Tüm dosyalar {OUTPUT_DIR} dizinine kaydedildi.")

In [ ]:
# Alternatif yükleme yöntemi: push_to_hub kullanarak
print("\n📤 Alternatif yükleme yöntemi (push_to_hub)...")

try:
    # Dataset'i doğrudan push_to_hub ile yükle
    hf_dataset.push_to_hub(
        repo_id=REPO_ID,
        token=HF_TOKEN,
        commit_message="Upload Turkish 200K clean dataset",
        private=False  # Public repo için False
    )
    
    print(f"✅ Veri seti push_to_hub ile başarıyla yüklendi!")
    print(f"🔗 Veri setinize erişin: https://huggingface.co/datasets/{REPO_ID}")
    
    # Kullanım örneği
    print("\n📚 Veri setinizi kullanmak için:")
    print(f"```python")
    print(f"from datasets import load_dataset")
    print(f"dataset = load_dataset('{REPO_ID}')")
    print(f"print(dataset['train'][0])")
    print(f"```")
    
except Exception as e:
    print(f"❌ push_to_hub hatası: {e}")
    print("Token ve repository adını kontrol edin.")

In [ ]:
# Veri setini Hugging Face'e yükle
from huggingface_hub import HfApi, create_repo, upload_folder
import shutil

print(f"\n🚀 Veri seti Hugging Face'e yükleniyor: {REPO_ID}")

try:
    api = HfApi()
    
    # 1. Repository oluştur (yoksa)
    try:
        create_repo(
            repo_id=REPO_ID,
            repo_type="dataset",
            private=False,  # Public yapmak için False, private için True
            token=HF_TOKEN
        )
        print(f"✅ Repository oluşturuldu: {REPO_ID}")
    except Exception as e:
        if "already exists" in str(e).lower():
            print(f"ℹ️ Repository zaten mevcut: {REPO_ID}")
        else:
            raise e
    
    # 2. Yüklenecek dosyaları hazırla
    upload_dir = os.path.join(OUTPUT_DIR, 'hf_upload')
    os.makedirs(upload_dir, exist_ok=True)
    
    # Dataset dosyalarını kopyala
    dataset_dir = os.path.join(OUTPUT_DIR, 'turkish_200k_dataset')
    if os.path.exists(dataset_dir):
        # Dataset klasörünü kopyala
        for item in os.listdir(dataset_dir):
            src = os.path.join(dataset_dir, item)
            dst = os.path.join(upload_dir, item)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
            else:
                shutil.copytree(src, dst, dirs_exist_ok=True)
    
    # README'yi kopyala
    shutil.copy2(readme_path, os.path.join(upload_dir, 'README.md'))
    
    # 3. Dosyaları yükle
    print("📤 Dosyalar yükleniyor...")
    upload_folder(
        folder_path=upload_dir,
        repo_id=REPO_ID,
        repo_type="dataset",
        token=HF_TOKEN,
        commit_message="Initial dataset upload: Turkish 200K clean dataset"
    )
    
    print(f"✅ Veri seti başarıyla yüklendi!")
    print(f"🔗 Veri setinize erişin: https://huggingface.co/datasets/{REPO_ID}")
    
    # Temizlik
    shutil.rmtree(upload_dir)
    
except Exception as e:
    print(f"❌ Yükleme hatası: {e}")
    print("Lütfen token'ınızı ve internet bağlantınızı kontrol edin.")

In [ ]:
# README.md oluştur
readme_content = f"""---
language:
- tr
license: apache-2.0
size_categories:
- 100K<n<1M
task_categories:
- text-generation
- text2text-generation
pretty_name: Turkish Clean Dataset 200K
tags:
- turkish
- nlp
- clean-data
- high-quality
dataset_info:
  features:
  - name: text
    dtype: string
  - name: quality_score
    dtype: float32
  - name: length
    dtype: int32
  - name: word_count
    dtype: int32
  splits:
  - name: train
    num_examples: {len(final_dataset)}
---

# 🇹🇷 Turkish Clean Dataset 200K

## Veri Seti Hakkında

Bu veri seti, Türkçe NLP modellerinin eğitimi için hazırlanmış yüksek kaliteli 200.000 metin örneği içerir.

### 📊 Özellikler

- **Toplam Örnek Sayısı**: {len(final_dataset):,}
- **Ortalama Metin Uzunluğu**: {np.mean(lengths):.0f} karakter
- **Ortalama Kelime Sayısı**: {np.mean(word_counts):.0f} kelime
- **Ortalama Kalite Skoru**: {np.mean(quality_scores):.2f}/100
- **Min Kalite Skoru**: {min(quality_scores):.2f}/100
- **Max Kalite Skoru**: {max(quality_scores):.2f}/100

### 🔧 Veri İşleme Süreci

1. **Kaynak Veri Setleri**:
   - muspdf dataset (500K örnek indirildi, kalite eşiği: %88)
   - CulturaX Turkish (300K örnek indirildi, kalite eşiği: %95)

2. **Temizleme İşlemleri**:
   - URL, email ve HTML etiketlerinin kaldırılması
   - Encoding problemlerinin düzeltilmesi (ftfy)
   - Fazla boşluk ve satır sonlarının temizlenmesi
   - Türkçe karakterlerin korunması

3. **Kalite Kontrol Kriterleri**:
   - Türkçe dil tespiti (langdetect)
   - Türkçe özel karakter oranı kontrolü
   - Metin uzunluğu değerlendirmesi (100-5000 karakter ideal)
   - Kelime çeşitliliği analizi
   - Noktalama işareti dengesi
   - Büyük/küçük harf oranı

### 📦 Veri Formatı

Veri seti 4 alan içerir:
- `text`: Temizlenmiş metin
- `quality_score`: Kalite skoru (0-100)
- `length`: Metin uzunluğu (karakter)
- `word_count`: Kelime sayısı

### 💻 Kullanım

```python
from datasets import load_dataset

# Veri setini yükle
dataset = load_dataset("{REPO_ID}")

# İlk örneği görüntüle
print(dataset['train'][0])

# Pandas DataFrame'e dönüştür
df = dataset['train'].to_pandas()
```

### 📈 Kalite Skoru Dağılımı

- Q1 (25%): {np.percentile(quality_scores, 25):.2f}
- Q2 (50%): {np.percentile(quality_scores, 50):.2f}
- Q3 (75%): {np.percentile(quality_scores, 75):.2f}

### 🎯 Kullanım Alanları

- Türkçe dil modelleri eğitimi
- Metin üretimi (text generation)
- Metin sınıflandırma
- Dil modelleme
- Transfer öğrenme

### 📄 Lisans

Apache 2.0

### 🤝 Katkıda Bulunma

Veri setini geliştirmek için önerilerinizi ve katkılarınızı bekliyoruz!

### 📮 İletişim

Sorularınız için issue açabilir veya tartışma bölümünü kullanabilirsiniz.
"""

# README'yi kaydet
readme_path = os.path.join(OUTPUT_DIR, 'README.md')
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)
print(f"✅ README.md oluşturuldu: {readme_path}")

In [ ]:
# Veri seti bilgilerini ayarla
DATASET_NAME = input("Veri seti adını girin (örn: turkish-200k-clean): ").strip()
if not DATASET_NAME:
    DATASET_NAME = "turkish-200k-clean"

USERNAME = input("Hugging Face kullanıcı adınızı girin: ").strip()
if not USERNAME:
    print("❌ Kullanıcı adı gerekli!")
else:
    REPO_ID = f"{USERNAME}/{DATASET_NAME}"
    print(f"\n📦 Veri seti repo ID: {REPO_ID}")

In [ ]:
# Hugging Face Hub'a giriş yap
from huggingface_hub import HfApi, login, create_repo
import getpass

print("🔐 Hugging Face'e giriş yapılıyor...")
print("Token'ınızı https://huggingface.co/settings/tokens adresinden alabilirsiniz.")
print("Token'ınız güvenli bir şekilde saklanacaktır.\n")

# Token'ı güvenli şekilde al
if IN_COLAB:
    from google.colab import userdata
    try:
        # Colab secrets'tan token'ı almayı dene
        HF_TOKEN = userdata.get('HF_TOKEN')
        print("✅ Token Colab secrets'tan alındı")
    except:
        # Manuel olarak iste
        HF_TOKEN = getpass.getpass("Hugging Face Token'ınızı girin: ")
else:
    # Önce çevre değişkeninden kontrol et
    HF_TOKEN = os.environ.get('HF_TOKEN')
    if not HF_TOKEN:
        HF_TOKEN = getpass.getpass("Hugging Face Token'ınızı girin: ")

# Giriş yap
try:
    login(token=HF_TOKEN, add_to_git_credential=True)
    print("✅ Hugging Face'e başarıyla giriş yapıldı!")
except Exception as e:
    print(f"❌ Giriş başarısız: {e}")
    print("Token'ınızı kontrol edip tekrar deneyin.")

## Adım 5: Veri Setini Hugging Face'e Yükle